## Tutorial on chemical reaction network (CRN) inference

### Simulating a CRN

Fisrt, we simulate a synthetic dataset based on the following CRN, i.e. our ground truth CRN:
$$ X_1\xrightarrow[]{k_{1}}X_2 \\ X_1+X_2\xrightleftharpoons[k_{13}]{k_{18}}X_3.$$
The subscripts of the reaction rate constants will be explained later when we introduce the library CRN.

The `@reaction_network` macro from `Catalyst.jl` enables us to programatically define CRNs. Note that package imports and first calls to functions take longer times to run due to compilation.

In [1]:
using Catalyst

In [2]:
true_rn = @reaction_network begin
	k1, X1 --> X2                # unidirectional reaction
	(k18, k13), X1 + X2 <--> X3	 # birectional reaction
end


Model ##ReactionSystem#236:
Unknowns (3): see unknowns(##ReactionSystem#236)
  X1(t)
  X2(t)
  X3(t)
Parameters (3): see parameters(##ReactionSystem#236)
  k1
  k18
  k13

`Catalyst.jl` features functionality to automatically derive the corresponding ordinary differential equation (ODE) system (assuming mass-action kinetics).

In [3]:
convert(ODESystem, true_rn)

Model ##ReactionSystem#236:
Equations (3):
  3 standard: see equations(##ReactionSystem#236)
Unknowns (3): see unknowns(##ReactionSystem#236)
  X1(t)
  X2(t)
  X3(t)
Parameters (3): see parameters(##ReactionSystem#236)
  k1
  k18
  k13

To simulate the dynamics of this ODE system, say over $t\in[0,10]$, we need to define and solve an `ODEProblem`. We will use the initial values $(X_1(0), X_2(0), X_3(0)) = (0,0,1)$ and set the reaction rate constants as $(k_{1},k_{13},k_{18})=(0,0,1)$.

In [4]:
using DifferentialEquations

In [6]:
t_span = (0., 10.); # time interval to solve on
x0map = [:X1 => 0., :X2 => 0., :X3 => 1.]; # initial conditions.
oprob = ODEProblem(true_rn, x0map, t_span, kmap);

UndefVarError: UndefVarError: `x0map` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### Defining a library CRN

### Parameter estimation

### Reaction network inference

### Hyperparameter tuning

### Alternative penalty function